In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images/Images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet_pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

1. CREATE A LABEL MAP

In [ ]:
labels = [
    {'name':'Hello', 'id':1},
    {'name':'Thanks', 'id':2},
    {'name':'Yes', 'id':3},
    {'name':'No', 'id':4},
    {'name':'I Love You', 'id':5},
]

with open(rf"{ANNOTATION_PATH}\label_map.pbtxt",'w')as f:
    for label in labels:
        f.write('item{ \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

2. Create TF Records

In [3]:
!python Tensorflow/scripts/generate_tfrecord.py -x {IMAGE_PATH + '/Train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/Train.record'}

!python Tensorflow/scripts/generate_tfrecord.py -x {IMAGE_PATH + '/Test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/Test.record'}


Traceback (most recent call last):
  File "c:\Users\KIIT0001\Sign_Language_Detection\Tensorflow\scripts\generate_tfrecord.py", line 115, in <module>
    tf_example = create_tf_example(group, args.image_dir, label_map_dict)
  File "c:\Users\KIIT0001\Sign_Language_Detection\Tensorflow\scripts\generate_tfrecord.py", line 85, in create_tf_example
    'image/object/class/label': dataset_util.int64_list_feature(classes),
  File "c:\Users\KIIT0001\anaconda3\envs\medibot\lib\site-packages\object_detection-0.1-py3.10.egg\object_detection\utils\dataset_util.py", line 30, in int64_list_feature
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
TypeError: None has type NoneType, but expected one of: int
Traceback (most recent call last):
  File "c:\Users\KIIT0001\Sign_Language_Detection\Tensorflow\scripts\generate_tfrecord.py", line 115, in <module>
    tf_example = create_tf_example(group, args.image_dir, label_map_dict)
  File "c:\Users\KIIT0001\Sign_Language_Detection\Tenso

3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


4. Copy Model Config to Training Folder

In [5]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [6]:

!copy "Tensorflow\\pre-trained-models\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\pipeline.config" "Tensorflow\\workspace\\models\\my_ssd_mobnet\\pipeline.config"



        1 file(s) copied.


5. Update Config For Transfer Learning

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'

In [9]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [10]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [11]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [12]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [13]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

6. Train the model

In [14]:
print("""python {}/research/object_detection/model_main_tf2.py --model_fir={}/{} --pipeline_config_path = {}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH,MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_fir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path = Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


In [15]:
import os
print("Train images:", os.listdir(IMAGE_PATH + '/Train'))
print("Test images:", os.listdir(IMAGE_PATH + '/Test'))

Train images: ['hello.4e68448d-322b-11f0-b739-d4d853ed0f11.jpg', 'hello.4e68448d-322b-11f0-b739-d4d853ed0f11.xml', 'hello.4f9c4d02-322b-11f0-85e0-d4d853ed0f11.jpg', 'hello.4f9c4d02-322b-11f0-85e0-d4d853ed0f11.xml', 'hello.50d28477-322b-11f0-bf64-d4d853ed0f11.jpg', 'hello.50d28477-322b-11f0-bf64-d4d853ed0f11.xml', 'hello.5204db3b-322b-11f0-9f36-d4d853ed0f11.jpg', 'hello.5204db3b-322b-11f0-9f36-d4d853ed0f11.xml', 'hello.559d1db4-322b-11f0-b60d-d4d853ed0f11.jpg', 'hello.559d1db4-322b-11f0-b60d-d4d853ed0f11.xml', 'hello.56cff6ed-322b-11f0-988d-d4d853ed0f11.jpg', 'hello.56cff6ed-322b-11f0-988d-d4d853ed0f11.xml', 'hello.5a68cc87-322b-11f0-af2e-d4d853ed0f11.jpg', 'hello.5a68cc87-322b-11f0-af2e-d4d853ed0f11.xml', 'hello.5b9b4da7-322b-11f0-98a3-d4d853ed0f11.jpg', 'hello.5b9b4da7-322b-11f0-98a3-d4d853ed0f11.xml', 'hello.5ccd9756-322b-11f0-8449-d4d853ed0f11.jpg', 'hello.5ccd9756-322b-11f0-8449-d4d853ed0f11.xml', 'hello.5e003453-322b-11f0-a97a-d4d853ed0f11.jpg', 'hello.5e003453-322b-11f0-a97a-d4d8

In [16]:
import pandas as pd
csv_path = f"{ANNOTATION_PATH}/train_labels.csv"
df = pd.read_csv(csv_path)
print("Missing values:\n", df.isnull().sum())
print("Unique classes:", df['class'].unique())


FileNotFoundError: [Errno 2] No such file or directory: 'Tensorflow/workspace/annotations/train_labels.csv'